# Create .PNG images of all timesteps in IDX firesmoke dataset

## Import necessary libraries

In [ ]:
# for numerical work
import numpy as np

# for accessing file system
import os

# for downloading latest firesmoke netCDF
import requests

# for loading netcdf files, for metadata
import xarray as xr
# for connecting OpenVisus framework to xarray
# from https://github.com/sci-visus/openvisuspy, 
from openvisuspy.xarray_backend import OpenVisusBackendEntrypoint

# Used for processing netCDF time data
import time
import datetime

# Used for indexing via metadata
import pandas as pd

# for plotting
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import cartopy.crs as ccrs

# for exporting the dictionary of issue files at the end of notebook
import pickle

# Stores the OpenVisus cache in the local direcrtory 
import os
os.environ["VISUS_CACHE"]="./visus_cache_can_be_erased"

# Accessory, used to generate progress bar for running for loops
# from tqdm.notebook import tqdm
# import ipywidgets
# import jupyterlab_widgets
from tqdm import tqdm

from OpenVisus import *

### In this section, we load our data using `xr.open_dataset`.

In [ ]:
# path to tiny NetCDF
url = 'https://github.com/sci-visus/NSDF-WIRED/raw/main/data/firesmoke_metadata.nc'

# # Download the file using requests
# response = requests.get(url)
local_netcdf = 'firesmoke_metadata.nc'
# with open(local_netcdf, 'wb') as f:
#     f.write(response.content)
    
# open tiny netcdf with xarray and OpenVisus backend
ds = xr.open_dataset(local_netcdf, engine=OpenVisusBackendEntrypoint)

## Calculate derived metadata using original metadata above to create coordinates
### This is required to allow for indexing of data via metadata

#### Calculate latitude and longitude grid

In [ ]:
# Get metadata to compute lon and lat
xorig = ds.XORIG
yorig = ds.YORIG
xcell = ds.XCELL
ycell = ds.YCELL
ncols = ds.NCOLS
nrows = ds.NROWS

longitude = np.linspace(xorig, xorig + xcell * (ncols - 1), ncols)
latitude = np.linspace(yorig, yorig + ycell * (nrows - 1), nrows)

#### Using calculated latitude and longitude, create coordinates allowing for indexing data using lat/lon

In [ ]:
# Create coordinates for lat and lon (credit: Aashish Panta)
ds.coords['lat'] = ('ROW', latitude)
ds.coords['lon'] = ('COL', longitude)

# Replace col and row dimensions with newly calculated lon and lat arrays (credit: Aashish Panta)
ds = ds.swap_dims({'COL': 'lon', 'ROW': 'lat'})

##### Return an array of the tflags as pandas timestamps

In [ ]:
def parse_tflag(tflag):
    """
    Return the tflag as a datetime object
    :param list tflag: a list of two int32, the 1st representing date and 2nd representing time
    """
    # obtain year and day of year from tflag[0] (date)
    date = int(tflag[0])
    year = date // 1000 # first 4 digits of tflag[0]
    day_of_year = date % 1000 # last 3 digits of tflag[0]

    # create datetime object representing date
    final_date = datetime.datetime(year, 1, 1) + datetime.timedelta(days=day_of_year - 1)

    # obtain hour, mins, and secs from tflag[1] (time)
    time = int(tflag[1])
    hours = time // 10000 # first 2 digits of tflag[1]
    minutes = (time % 10000) // 100 # 3rd and 4th digits of tflag[1] 
    seconds = time % 100  # last 2 digits of tflag[1]

    # create final datetime object
    full_datetime = datetime.datetime(year, final_date.month, final_date.day, hours, minutes, seconds)
    return full_datetime

In [ ]:
# get all tflags
tflag_values = ds['TFLAG'].values

# to store pandas timestamps
timestamps = []

# convert all tflags to pandas timestamps, store in timestamps list
for tflag in tflag_values:
    timestamps.append(pd.Timestamp(parse_tflag(tflag[0])))

# check out the first 3 timestamps
timestamps[0:3]

In [ ]:
# set coordinates to each timestep with these pandas timestamps
ds.coords['time'] = ('time', timestamps)

### Load the ECCC data, generated as in `eccc_data_clean.ipynb`

In [ ]:
df_eccc = pd.read_csv('PM25_2021_2022.csv')
eccc_lons = df_eccc['Longitude//Longitude'].values
eccc_lats = df_eccc['Latitude//Latitude'].values
eccc_dates = df_eccc['Date//Date'].values

### Create set of all hours to query for

In [ ]:
all_dates = []
for d in eccc_dates:
    for i in range(24):
        all_dates.append(pd.Timestamp(f'{d} {i}:00:00'))

In [ ]:
for i in enumerate(all_dates[0:5]):
    print(i)

### Get IDX forecast at points closest to ECCC points

In [ ]:
# find nearest lat, lon values in idx to eccc lat, lon values
data_resolution = 0
ds_eccc_pts = ds.loc[dict(resolution=data_resolution)].sel(
    lat=eccc_lats, lon=eccc_lons, method='nearest')

# loop thru the coords, populate array with coord vals and pm25 vals
idx_coords = np.column_stack([ds_eccc_pts.lat.values, ds_eccc_pts.lon.values])

In [ ]:
idx_lats = idx_coords[:, 0]
idx_lons = idx_coords[:, 1]

## Create the frames

In [ ]:
val = ds_idx['PM25'].sel(time=curr_date, lat=lat, 
            lon=lon, resolution=data_resolution).values

In [ ]:
oh = ds['PM25'].sel(time=all_dates[2000], lat=idx_lats, lon=idx_lons, resolution=0)

In [ ]:
oh.values

In [ ]:
np.shape()

In [ ]:
# set parameters for creating visualization of each timestep with matplotlib
my_norm = "log"
my_extent = [np.min(eccc_lons), np.max(eccc_lons), np.min(eccc_lats), np.max(eccc_lats)]
my_aspect = 'auto'
my_origin = 'lower'
my_cmap = 'autumn'
my_vmin = 1e-13
my_vmax = 10e4
fig_w, fig_h = 15, 6
issue_files = {}
full_data = False
if full_data:
    save_dir = "/usr/sci/scratch_nvme/arleth/dump/idx_vs_eccc_frames/full_data/"
else:
    save_dir = "/usr/sci/scratch_nvme/arleth/dump/idx_vs_eccc_frames/point_data/"


def create_frame_catch_issues(frame_date_tuple):
    # frame number to save PNG as and date to visualize
    frame_num = frame_date_tuple[0]
    date = frame_date_tuple[1]

    # # create visualization using matplotlib and cartopy geography lines
    # try: # visualize data if it's available
    data_array_at_time = None

    # set up visualization
    my_fig, my_plt = plt.subplots(figsize=(fig_w, fig_h), subplot_kw=dict(projection=ccrs.PlateCarree()))
    my_plt.coastlines()
    my_plt.gridlines(draw_labels=True)
    my_fig.suptitle(f'Ground level concentration of PM2.5 microns and smaller {date}\n')

    if full_data:# get PM25 values and provide 4 values, the colons mean select all lat and lon indices
        data_array_at_time = ds['PM25'].loc[date, :, :, data_resolution]
        plot = my_plt.imshow(data_array_at_time, extent=my_extent,
                         aspect=my_aspect, origin=my_origin, cmap=my_cmap,
                         norm=my_norm, vmax=my_vmax, vmin=my_vmin)
    else:
        data_array_at_time = ds['PM25'].loc[date, :, :, data_resolution]
        my_plt.set_extent(my_extent, crs=ccrs.PlateCarree())
        my_plt.set_aspect('auto')
        # get the values for the given hour and get latitudes and longitudes for plotting
        curr_vals = data_array_at_time.values.flatten()
        curr_lats = eccc_lats
        curr_lons = eccc_lons

        plot = my_plt.scatter(curr_lons, curr_lats, c=curr_vals, cmap=my_cmap,
                                 norm=my_norm, s=5, transform=ccrs.PlateCarree(),
                                 vmin=my_vmin, vmax=my_vmax)

    my_fig.colorbar(plot, location='right', label='ug/m^3')
    # add caption showing this is from IDX dataset
    plt.text(0.5, -0.1, 'IDX Data', ha='center', va='center', transform=my_plt.transAxes)

    # save visualization as a .PNG to our folder
    plt.savefig(save_dir + "frames%05d.png" % frame_num, dpi=280)
    plt.close(my_fig);  # close the figure after saving
    matplotlib.pyplot.close()
    # my_plt.show()
    # except: # return empty image otherwise
    #     fig, ax = plt.subplots(figsize=(fig_w, fig_h))
    #     ax.axis('off')
    #     plt.text(.5, .5, 'IDX Data UNAVAILABLE', fontsize=20, horizontalalignment='center',
    #  verticalalignment='center',)
    #     # save visualization as a .PNG to our folder
    #     plt.savefig(save_dir + "frames%010d.png" % frame_num, dpi=280)
    #     plt.close(fig);  # close the figure after saving
    #     # plt.show()
    #     return date

In [ ]:
proc_lim = 10
# create frames, capturing issues 
with multiprocessing.Pool(processes=proc_lim) as pool:
    # Start a timer to measure how long the conversion takes
    start_time = time.time()
    print('starting')
    issues = pool.map(create_frame_catch_issues, enumerate([all_dates[2000]]))
    print('done!')
    # End the timer and print the elapsed time
    end_time = time.time()
    print(f'Total elapsed time: {end_time - start_time}')

In [ ]:
# save 'issue_files' to review
with open('new_idx_issues.pkl', 'wb') as f:
    pickle.dump(issue_files, f)

In [ ]:
with open('new_idx_issues.pkl', 'rb') as f:
    new_idx_issues = pickle.load(f)
print(f'len of new_idx_issues.pkl = {len(new_idx_issues)}')

#### old basemap attempt

Get latitude and longitude values to use basemap to plot them with corresponding PM2.5 values

In [ ]:
# # populate these with all coordinates and values at current tstep
# # get PM25 values and provide 4 values, the colons mean select all lat and lon indices
# curr_ds = ds['PM25'].loc[0, :, :, 0]
# arr_size = ds.sizes['lat'] * ds.sizes['lon']
# lons = np.zeros(arr_size)
# lats = np.zeros(arr_size)
# vals = np.zeros(arr_size)
# c = 0

# for i in np.arange(ds.sizes['lat']):
#     for j in np.arange(ds.sizes['lon']):
#         lats[c] = curr_ds.lat.values[i]
#         lons[c] = curr_ds.lon.values[j]
#         vals[c] = curr_ds.values[i][j]
#         c += 1
# a = lons
# b = lats
# c = vals

In [ ]:
# data_resolution = 0
# save_dir = "/usr/sci/scratch_nvme/arleth/dump/idx_vs_eccc_frames/"
# issue_files = {}

# def create_frame_catch_issues(tstep):
#     # get PM25 values and provide 4 values, the colons mean select all lat and lon indices
#     curr_ds = ds['PM25'].loc[tstep, :, :, data_resolution]
    
#     # populate these with all coordinates and values at current tstep
#     arr_size = ds.sizes['lat'] * ds.sizes['lon']
#     lons = np.zeros(arr_size)
#     lats = np.zeros(arr_size)
#     vals = np.zeros(arr_size)
#     c = 0
    
#     for i in np.arange(ds.sizes['lat']):
#         for j in np.arange(ds.sizes['lon']):
#             lats[c] = curr_ds.lat.values[i]
#             lons[c] = curr_ds.lon.values[j]
#             vals[c] = curr_ds.values[i][j]
#             c += 1
#     # # create visualization using matplotlib and cartopy geography lines, 
#     # # catch exceptions accordingly
#     # try:
#     # use basemap to plot values: https://basemaptutorial.readthedocs.io/en/latest/plotting_data.html#scatter
#     # use `cyl` project: https://matplotlib.org/basemap/stable/users/cyl.html
#     # set parameters: https://basemaptutorial.readthedocs.io/en/latest/basemap.html
#     m = Basemap(projection='cyl', llcrnrlat=np.min(lats),urcrnrlat=np.max(lats),
#             llcrnrlon=np.min(lons),urcrnrlon=np.max(lons), lat_0=51, lon_0=-106, resolution='l',
#             fix_aspect=False, area_thresh=1e6)

#     # Draw map features
#     m.drawcoastlines()
#     # m.drawparallels(np.arange(45.,66.,5.),labels=[1,1,1,1]) # draw parallels
#     # m.drawmeridians(np.arange(-120.,-59.,20.),labels=[1,1,1,1]) # draw parallels

#     # Convert lat/lon to map coordinates for Basemap scatter plot
#     x, y = m(lons, lats)

#     # Plot the lats and lons
#     m.scatter(x, y, marker=',', s=.001, c=vals, cmap='hot')
#     plt.show()
#     # except:
#     #     t = pd.Timestamp(parse_tflag(ds.TFLAG[tstep][0]))
#     #     print(f"issue! {t}")
#     #     return t, curr_values